In [95]:
from query_vroom import *
from query_osrm import OsrmQuery
import pandas as pd
import numpy as np
import json
from clustering import DBScanClusterer

# with open(r"C:\Users\isaac.li\source\repos\route - admin_frontend\data.json") as f:
with open(r"C:\Users\clement.lork\Documents\Sembwaste_Routing\swm_route_optimization\azure_repos\admin_frontend\data.json") as f:
  data = json.loads(f.read())

### Convert to DataFrame

In [96]:
data.keys()

dict_keys(['routesToBreak', 'routesToOptimize', 'pointsToAssign', 'vehiclesInvolved', 'depotData', 'solverConfig', 'allVehicleTypes'])

In [97]:
data["allVehicleTypes"]

[{'id': 0, 'Name': '4x2', 'Capacity': 8000, 'Range': 300000},
 {'id': 1, 'Name': '6x4', 'Capacity': 12000, 'Range': 300000},
 {'id': 2, 'Name': '8x2', 'Capacity': 16000, 'Range': 300000}]

In [98]:
solverConfig = data["solverConfig"]
# solverConfig['useClusters'] = True

points = data["pointsToAssign"]
fleet = data["vehiclesInvolved"]
vtypes = data['allVehicleTypes']
routebreak = data['routesToBreak']
routeopt = data['routesToOptimize']
depots = data['depotData']

ips = [{'id': 0, 'Name': 'IPs', 'lat': 1.29607, 'long': 103.62173}]

In [99]:
solverConfig 

{'mode': 'Breakroute', 'useClusters': True}

In [100]:
depots

[{'id': 0, 'Name': 'Tuas', 'lat': 1.3344, 'long': 103.64579}]

In [101]:
points = pd.DataFrame.from_dict(points)
fleet = pd.DataFrame.from_dict(fleet)
vtypes = pd.DataFrame.from_dict(vtypes)
routebreak = pd.DataFrame.from_dict(routebreak)
routeopt = pd.DataFrame.from_dict(routeopt)
depots = pd.DataFrame.from_dict(depots)
ips = pd.DataFrame.from_dict(ips)

In [102]:
total_routes = pd.concat([routeopt,routebreak])

In [104]:
fleet

,id,Name,Vehicle_Type,Route_ID,Max_Trips,Manpower,Depot,Break_Time,Current_Lat,Current_Long,Current_Tonnage,Start_Time,End_Time
0,0,XE6239D,2,0,3,2,0,12,1.3344,103.64579,0,00:00,00:00
1,1,XE5992T,1,1,2,3,0,12,1.3344,103.64579,0,00:00,00:00
2,3,XE5988G,1,3,3,3,0,12,1.3344,103.64579,0,00:00,00:00
3,4,XE6059G,2,4,3,2,0,12,1.3344,103.64579,0,00:00,00:00
4,5,XE6193B,2,5,3,2,0,12,1.3344,103.64579,0,00:00,00:00
5,6,XE6015H,1,6,3,2,0,12,1.3344,103.64579,0,00:00,00:00
6,7,XE6131E,2,7,3,2,0,12,1.3344,103.64579,0,00:00,00:00
7,8,XE6350P,2,8,3,2,0,12,1.3344,103.64579,0,00:00,00:00


In [103]:
# fleet["Route_ID"] = fleet["Route_ID"].astype("int")
# fleet["Manpower"] = fleet["Manpower"].astype("int")
# fleet["vehicleCapacity"] = fleet["vehicleCapacity"].astype("int")

KeyError: 'vehicleCapacity'

In [63]:
endpoint = "http://localhost:5001"
querycar = OsrmQuery(endpoint, 'car')
route_response = querycar.find_route(1.2984382,103.8047524,1.2980271,103.8050503,u_turn_allowed=True)
route_response["routes"][0]["duration"]/60

0.26666666666666666

In [64]:
vtypes2str = dict(zip(vtypes.id.tolist(),vtypes.Name.tolist()))
vtypes2id = dict(zip(vtypes.Name.tolist(),vtypes.id.tolist()))
vnames2cap = dict(zip(vtypes.Name.tolist(),vtypes.Capacity.tolist()))

In [65]:
# convert Vehicle_Type back to string
fleet["Vehicle_Type"] = fleet["Vehicle_Type"].apply(lambda x: vtypes2str[x])
fleet["Capacity"] = fleet["Vehicle_Type"].apply(lambda x: vnames2cap[x])

In [66]:
# check if a vehicle is servicing point.route_id, if true set allowed vehicle as the vehicle id

rid2vid = dict(zip(fleet.Route_ID.tolist(),fleet.id.tolist()))
vt2vid = dict(zip(fleet.Vehicle_Type.tolist(),fleet.id.tolist()))
vid2mp = dict(zip(fleet.id.tolist(),fleet.Manpower.tolist()))

vt2vid  = {}

for idx, vtype in enumerate(fleet['Vehicle_Type']):
    if vtype not in vt2vid :
        vt2vid[vtype] = []
        vt2vid[vtype].append(fleet["id"][idx])
    else:
        vt2vid[vtype].append(fleet["id"][idx])

print(rid2vid)
print(vt2vid)
print(vid2mp)


{0: 0, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8}
{'8x2': [0, 4, 5, 7, 8], '6x4': [2, 3, 6]}
{0: 3, 2: 3, 3: 3, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2}


In [67]:
points["Manpower"].unique()

array([2, 3], dtype=int64)

### Find vehicle skils before clustering

In [68]:
vehicle_skills = {}
for vid in fleet.id:
    vehicle_skills[vid] = []

for idx, p in points.iterrows():
    # if route_id of points is inside routesInvolved, append p.id to vehicle
    rid = p["Route_ID"]
    allowed_veh = []
    if rid in routeopt.id.tolist():
        vid = rid2vid[rid]
        vehicle_skills[vid].append(p['id'])
        points.loc[idx,"Allowed_Veh"] = str(vid)
    else:
        # if vehicles does not have assigned routes, fall back to checking vehicle_type_allowed tags
        allowed_veh_list = []
        for vt in p["Vehicle_Type_Allowed"].split(","):
            if vt in vt2vid.keys():
                # allowed_veh_list += [str(i) for i in vt2vid[vt]]
                for vid in vt2vid[vt]:
                    # check if vehicle support manpower of point
                    if p["Manpower"] <= vid2mp[vid]:
                        vehicle_skills[vid].append(p['id'])
                        allowed_veh_list += [str(vid)]


        allowed_veh = ",".join(sorted(allowed_veh_list))
        points.loc[idx,"Allowed_Veh"] = allowed_veh

In [69]:
points["Allowed_Veh"].unique()

try:
    assert ('' not in points["Allowed_Veh"].unique())
except AssertionError as e:
    print(e)

In [70]:
# import json
# import pandas as pd

# def read_data_from_js(file_path, string_to_split):
#     # Read the JavaScript file
#     with open(file_path, 'r') as f:
#         js_content = f.read()

#     # Extract the points_data array from the JavaScript content
#     points_data_str = js_content.split(string_to_split)[1].split(';')[0]

#     # Convert the points_data string to a Python list of dictionaries
#     points_data = json.loads(points_data_str)

#     return points_data
# depots_map = { 0 : "Tuas" }
# veh_types_map = { 0 : "4x2", 1 : "6x4", 2 : "8x2" }
# routes_map = { 0 : "R16X", 1 : "R17", 2 : "R18", 3 : "R20X", 4 : "R22", 5 :"R23X", 6 : "R24", 7 : "R25X", 8 : "R26X"}

# depots_data_df = pd.DataFrame(read_data_from_js(r'C:\Users\isaac.li\source\repos\route - admin_frontend\static\js\data\depots_data.js', 'var depots_data ='))
# vehicle_types_data_df = pd.DataFrame(read_data_from_js(r'C:\Users\isaac.li\source\repos\route - admin_frontend\static\js\data\vehicle_types_data.js', 'var vehicle_types_data ='))

# routes_data_df = pd.DataFrame(read_data_from_js(r'C:\Users\isaac.li\source\repos\route - admin_frontend\static\js\data\routes_data.js', 'var routes_data ='))
# routes_data_df['Depot'] = routes_data_df['Depot'].map(depots_map)
# routes_data_df['Vehicle_Type'] = routes_data_df['Vehicle_Type'].map(veh_types_map)

# vehicles_data_df = pd.DataFrame(read_data_from_js(r'C:\Users\isaac.li\source\repos\route - admin_frontend\static\js\data\vehicles_data.js', 'var vehicles_data ='))
# vehicles_data_df['Depot'] = vehicles_data_df['Depot'].map(depots_map)
# vehicles_data_df['Vehicle_Type'] = vehicles_data_df['Vehicle_Type'].map(veh_types_map)
# vehicles_data_df['Route_ID'] = vehicles_data_df['Route_ID'].map(routes_map)

# excel_file_path = "sample_fleet_excel.xlsx"

# # Create a Pandas Excel writer using XlsxWriter as the engine
# with pd.ExcelWriter(excel_file_path) as writer:
#     depots_data_df.to_excel(writer, sheet_name='Depots', index=False)
#     vehicle_types_data_df.to_excel(writer, sheet_name='Vehicle Types', index=False)
#     routes_data_df.to_excel(writer, sheet_name='Routes', index=False)
#     vehicles_data_df.to_excel(writer, sheet_name='Vehicles', index=False)


### Cluster Points again if using clustering

In [71]:
cluster_map = dict(zip(points.Network_Cluster.unique().tolist(),list(range(len(points.Network_Cluster.unique())))))
points["Network_Cluster"] = points["Network_Cluster"].apply(lambda x: cluster_map[x])

In [72]:
solverConfig

{'mode': 'breakroute', 'useClusters': False}

In [73]:

veh_key = "Allowed_Veh"
dbscan = DBScanClusterer(points, querycar)
if solverConfig['useClusters'] == False:
    dbscan.dbs_cluster(eps=1)
dbscan.split_by_vehicle(split=True,veh_key=veh_key)
dbscan.split_by_timing(find_overlaps=False)
# dbscan.split_by_tonnage_recursive(tonnage_threshold=2000, initial_eps = 500, descent=100, column='Manpower',condition=3)
# dbscan.split_by_tonnage_recursive(tonnage_threshold=6000, initial_eps=500, descent=50)
cluster_midpoints, cluster_load, cluster_tw, cluster_restrictions, cluster_service_time, cluster_manpower = dbscan.find_cluster_characteristics()
points = dbscan.points

100%|██████████| 499/499 [00:03<00:00, 151.19it/s]


/table/v1/car/103.80332,1.297665;103.801706,1.299297;103.801294,1.298164;103.802995,1.297086;103.803496,1.294778;103.806086,1.293593;103.80412,1.295495;103.802664,1.291689;103.802472,1.29026;103.802284,1.297919;103.800725,1.296141;103.800915,1.291346;103.806718,1.29029;103.807823,1.287883;103.815047,1.289347;103.813202,1.287386;103.816379,1.286437;103.815199,1.285433;103.815807,1.28685;103.816388,1.287221;103.816903,1.286638;103.817013,1.2879;103.815593,1.288235;103.815524,1.288029;103.816096,1.288295;103.81649,1.28885;103.81703,1.28894;103.817579,1.288549;103.819357,1.287941;103.81935,1.287302;103.819121,1.28669;103.818874,1.28727;103.818841,1.285929;103.81865,1.285942;103.810687,1.292972;103.811905,1.292993;103.811382,1.294159;103.810932,1.294239;103.811594,1.293086;103.811608,1.293685;103.812281,1.293695;103.812345,1.29412;103.811322,1.29288;103.810391,1.29442;103.81033,1.29398;103.810534,1.295311;103.810399,1.295493;103.809913,1.295565;103.809425,1.295635;103.82325,1.290131;103.807

### Vehicle skills for vehicle after clustering

In [74]:
vehicle_skills = {}
for vid in fleet.id:
    vehicle_skills[vid] = []

for idx in range(0,len(cluster_midpoints)):
    restrictions = cluster_restrictions[idx].split(',')
    for r in restrictions:
        vehicle_skills[int(r)].append(idx)
    

### Add Start Point and End Points

In [75]:
tuas_osmid, tuas_latlong = querycar.find_nearest(1.33469, 103.64579)
ip_osmid, ip_latlong = querycar.find_nearest(1.29607, 103.62173)
work_timewindows = [7*60*60,24*60*60]

cluster_latlongs = cluster_midpoints.copy()
cluster_latlongs.append(tuas_latlong)
print("tuas@",len(cluster_latlongs)-1)
depot_idx = len(cluster_latlongs)-1
cluster_latlongs.append(ip_latlong)
print("ip@",len(cluster_latlongs)-1)
ip_idx = len(cluster_latlongs)-1

tuas@ 496
ip@ 497


### Calculate distance matrix

In [76]:
dist_matrix, time_matrix = querycar.find_matrix(cluster_latlongs)
time_matrix = time_matrix*60
# time_matrix[0:500,500] = time_matrix[0:500,500]+10*60*60
# time_matrix[500,0:500] = time_matrix[500,0:500]+10*60*60

dist_matrix = dist_matrix.astype("int64").tolist()
time_matrix = time_matrix.astype("int64").tolist()

/table/v1/car/103.80332,1.297665;103.801706,1.299297;103.801294,1.298164;103.802995,1.297086;103.803496,1.294778;103.806086,1.293593;103.80412,1.295495;103.802664,1.291689;103.802472,1.29026;103.802284,1.297919;103.800725,1.296141;103.800915,1.291346;103.806718,1.29029;103.807823,1.287883;103.815047,1.289347;103.813202,1.287386;103.816379,1.286437;103.815199,1.285433;103.815807,1.28685;103.816388,1.287221;103.816903,1.286638;103.817013,1.2879;103.815593,1.288235;103.815524,1.288029;103.816096,1.288295;103.81649,1.28885;103.81703,1.28894;103.817579,1.288549;103.819357,1.287941;103.81935,1.287302;103.819121,1.28669;103.818874,1.28727;103.818841,1.285929;103.81865,1.285942;103.810687,1.292972;103.811905,1.292993;103.811382,1.294159;103.810932,1.294239;103.811594,1.293086;103.811608,1.293685;103.812281,1.293695;103.812345,1.29412;103.811322,1.29288;103.810391,1.29442;103.81033,1.29398;103.810534,1.295311;103.810399,1.295493;103.809913,1.295565;103.809425,1.295635;103.82325,1.290131;103.807

### Add Vehicles

In [77]:
fleet

,id,Name,Vehicle_Type,Route_ID,Max_Trips,Manpower,Depot,Break_Time,Current_Lat,Current_Long,Current_Tonnage,Start_Time,End_Time,vehicleCapacity,vehicleRange,Capacity
0,0,XE6239D,8x2,0,3,3,0,12,1.3344,103.64579,0,00:00,00:00,16000,300000,16000
1,2,XE5596D,6x4,2,3,3,0,12,1.3344,103.64579,0,00:00,00:00,12000,300000,12000
2,3,XE5988G,6x4,3,3,3,0,12,1.3344,103.64579,0,00:00,00:00,12000,300000,12000
3,4,XE6059G,8x2,4,3,2,0,12,1.3344,103.64579,0,00:00,00:00,16000,300000,16000
4,5,XE6193B,8x2,5,3,2,0,12,1.3344,103.64579,0,00:00,00:00,16000,300000,16000
5,6,XE6015H,6x4,6,3,2,0,12,1.3344,103.64579,0,00:00,00:00,12000,300000,12000
6,7,XE6131E,8x2,7,3,2,0,12,1.3344,103.64579,0,00:00,00:00,16000,300000,16000
7,8,XE6350P,8x2,8,3,2,0,12,1.3344,103.64579,0,00:00,00:00,16000,300000,16000


In [78]:
vehicles = []
for idx, veh in fleet.iterrows():
    v = Vehicle(veh['id'], 'car', 
                start =None, start_index=int(depot_idx), 
                end =None, end_index=int(depot_idx), 
                capacity=[int(veh["Capacity"])], costs=None,
                skills = vehicle_skills[veh['id']],
                time_window=[7*60*60,19*60*60], max_tasks=None,
                max_travel_time=None,
                max_distance=300000).to_dict()
    vehicles.append(v)

In [79]:
data.keys()

dict_keys(['routesToBreak', 'routesToOptimize', 'pointsToAssign', 'vehiclesInvolved', 'depotData', 'solverConfig', 'allVehicleTypes'])

### Add pickup delivery (cluster)

In [80]:
shipments = []
for idx in range(0,len(cluster_midpoints)):
    amount = [cluster_load[idx]]
    skills = [idx]
    pickup_step = Shipment_Step(id=idx,
                                description = str("cluster")+str(idx),
                                location = None,
                                location_index = idx,
                                setup = None,
                                service = int(cluster_service_time[idx])*60,
                                time_windows = [[int(cluster_tw[idx][0])*60*60, int(cluster_tw[idx][1])*60*60]]).to_dict()
    delivery_step = Shipment_Step(id=len(cluster_midpoints)+2+idx,
                                  description = "IP",
                                  location= None,
                                  location_index= ip_idx,
                                  setup = 20*60,
                                  service= None,
                                  time_windows = [[7*60*60, 24*60*60]]).to_dict()
    ship = Shipment(pickup_step=pickup_step, delivery_step=delivery_step, amount=amount, skills=skills, priority=0).to_dict()
    shipments.append(ship)


    matrix = Matrix(duration_matrix = time_matrix, distance_matrix=None, cost_matrix=None).to_dict()


In [81]:
data = sendData(jobs=[],shipments=shipments,vehicles=vehicles,matrices=matrix).to_dict()
data = json.dumps(data, indent=2, default=int)
data = json.loads(data)
output = solve(data)

In [82]:
output["summary"]

{'cost': 100680,
 'routes': 8,
 'unassigned': 38,
 'delivery': [212730],
 'amount': [212730],
 'pickup': [212730],
 'setup': 27600,
 'service': 179340,
 'duration': 100680,
 'waiting_time': 0,
 'priority': 0,
 'violations': [],
 'computing_times': {'loading': 8, 'solving': 23594, 'routing': 0}}

In [83]:
def sec2time(seconds):
    hours = seconds // 3600
    remaining_seconds = seconds % 3600
    minutes = remaining_seconds // 60
    return '{:02d}:{:02d}'.format(hours,minutes)

In [84]:
total_assignment = {}

for k in range(0,len(output['routes'])):
    assignment = {}
    time = []
    route = []
    for i in output['routes'][k]["steps"]:
        route.append(i["location_index"])
        time.append(sec2time(i['arrival']))
        # if i["location_index"] == depot_idx:
        #     route.append("DP")
        # elif i["location_index"] > depot_idx:
        #     route.append("IP")
        # else:
        #     route.append(i["load"][0])
    route = [route[i] for i in range(len(route)) if i == 0 or route[i] != route[i - 1]]
    time = [time[i] for i in range(len(time)) if i == 0 or time[i] != time[i - 1]]
    assignment['time'] = time
    assignment['route'] = route
    total_assignment[fleet.loc[fleet["id"]==output['routes'][k]['vehicle'],"Name"].tolist()[0]] = assignment


In [85]:
from tsp import run_tsp

def get_cluster_latlongs(cluster_latlongs, cluster_id):
    if cluster_id>=len(cluster_latlongs):
        return cluster_latlongs[-1]
    else:
        return cluster_latlongs[cluster_id]

def find_order_tsp(points, cluster_latlongs, querycar, prev_cluster, current_cluster,next_cluster):
    start = get_cluster_latlongs(cluster_latlongs, prev_cluster)
    end = get_cluster_latlongs(cluster_latlongs, next_cluster)
    locations = points.loc[points["Network_Cluster"]==current_cluster,"Node_Latlong"].tolist()
    if len(locations)>1:
        # print(prev_cluster,current_cluster,next_cluster)
        #create node list
        node_list = [start]+locations+[end]
        #generate distance matrix from node list
        dist_matrix, time_matrix = querycar.find_matrix(node_list)
        #run tsp
        r = run_tsp(dist_matrix)[0:-1] # remove the addition of startpoint and endpoint
        r = [n-1 for n in r]
        out = points.loc[points["Network_Cluster"]==current_cluster]
        out = out.reset_index().iloc[r]
    else:
        out = points.loc[points["Network_Cluster"]==current_cluster]
    return out

In [86]:
for v in total_assignment:
    cluster_nodes = total_assignment[v]['route']
    expanded = []
    for idx, cluster in enumerate(cluster_nodes):
        if cluster == depot_idx:
            expanded.append("DP")
        elif cluster == ip_idx:
            expanded.append("IP")
        else:
            if idx>0 and idx<len(cluster_nodes):
                start = cluster_nodes[idx-1]
                current = cluster_nodes[idx]
                end = cluster_nodes[idx+1]
                try:
                    locations = find_order_tsp(points, cluster_latlongs, querycar, start, current, end)
                except Exception as e:
                    print(e)
                    locations = points[points["Network_Cluster"]==cluster]
                expanded+=locations.id.tolist()
    total_assignment[v]["points_id"]= expanded

/table/v1/car/103.807079,1.294976;103.82325,1.290131;103.82325,1.290131;103.813202,1.287386?annotations=distance,duration
200
Solver status:  1
/table/v1/car/103.838767,1.275757;103.839248,1.273877;103.839248,1.273881;103.840056,1.273812?annotations=distance,duration
200
Solver status:  1
/table/v1/car/103.809901,1.288747;103.810248,1.288748;103.810248,1.288748;103.809182,1.288593?annotations=distance,duration
200
Solver status:  1


In [87]:
unassigned= [ua for ua in output["unassigned"] if ua["type"] == "pickup"] 

expanded = []
for u in unassigned:
    locations = points[points["Network_Cluster"]==u['id']]
    expanded += locations.id.tolist()

total_assignment["unassigned"] = {}
total_assignment["unassigned"]["points_id"] = expanded

In [88]:
print(unassigned )

[{'id': 412, 'location_index': 412, 'type': 'pickup', 'description': 'cluster412'}, {'id': 388, 'location_index': 388, 'type': 'pickup', 'description': 'cluster388'}, {'id': 387, 'location_index': 387, 'type': 'pickup', 'description': 'cluster387'}, {'id': 386, 'location_index': 386, 'type': 'pickup', 'description': 'cluster386'}, {'id': 368, 'location_index': 368, 'type': 'pickup', 'description': 'cluster368'}, {'id': 367, 'location_index': 367, 'type': 'pickup', 'description': 'cluster367'}, {'id': 352, 'location_index': 352, 'type': 'pickup', 'description': 'cluster352'}, {'id': 351, 'location_index': 351, 'type': 'pickup', 'description': 'cluster351'}, {'id': 350, 'location_index': 350, 'type': 'pickup', 'description': 'cluster350'}, {'id': 349, 'location_index': 349, 'type': 'pickup', 'description': 'cluster349'}, {'id': 348, 'location_index': 348, 'type': 'pickup', 'description': 'cluster348'}, {'id': 291, 'location_index': 291, 'type': 'pickup', 'description': 'cluster291'}, {'i

In [89]:
routeopt

,id,Name,Depot,Vehicle_Type,Max_Trips,Manpower,Points_Count
0,0,R16X,0,2,3,2,52
1,2,R18,0,1,3,3,60
2,3,R20X,0,1,3,3,78
3,4,R22,0,2,3,2,41
4,5,R23X,0,2,3,2,20
5,6,R24,0,1,3,2,66
6,7,R25X,0,2,3,2,19
7,8,R26X,0,2,3,2,63


In [90]:
##make sure uid is 
points.set_index('id', inplace=True)
fleet.set_index('id', inplace=True)
vtypes.set_index('id', inplace=True)
depots.set_index('id', inplace=True)
ips.set_index('id', inplace=True)
total_routes.set_index('id', inplace=True)


In [91]:
output = {}

output["points"] = points.to_dict() 
output["fleet"] = fleet.to_dict()
output['vtypes'] = vtypes.to_dict()
output["depots"] = depots.to_dict()
output["ips"] = ips.to_dict()
output["assignment"] = total_assignment
output["total_routes"] = total_routes.to_dict()


output["assignment"] = total_assignment

In [92]:
import json
with open(r"C:\Users\clement.lork\Documents\Sembwaste_Routing\swm_route_optimization\azure_repos\admin_frontend\output.json", 'w') as f:
    json.dump(output, f)

In [93]:
import numpy as np
import folium
import matplotlib
import ast

# Add arrows along the PolyLine to indicate direction
def add_arrows(m, line, color='black', size=10):
    locations = line.locations
    latitudes, longitudes = zip(*locations)

    for i in range(0,len(latitudes) - 1,5):
        arrow = folium.Marker(
            location=[latitudes[i], longitudes[i]],
            icon=folium.DivIcon(
                icon_size=(size, size),
                icon_anchor=(size / 2, size / 2),
                html='<div style="font-size: {}px; color: {}; transform: rotate({}deg);">&#10148;</div>'.format(
                    size, color, calculate_bearing(latitudes[i], longitudes[i], latitudes[i+1], longitudes[i+1])
                )
            )
        )
        arrow.add_to(m)

# Function to calculate bearing between two points
def calculate_bearing(lat1, lon1, lat2, lon2):
    delta_lon = lon2 - lon1
    y = np.sin(np.radians(delta_lon)) * np.cos(np.radians(lat2))
    x = np.cos(np.radians(lat1)) * np.sin(np.radians(lat2)) - \
        np.sin(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.cos(np.radians(delta_lon))
    initial_bearing = np.arctan2(y, x)
    compass_bearing = (np.degrees(initial_bearing) + 270) % 360
    return compass_bearing

def find_route(start,end):
    route_response = querycar.find_route(start[0],start[1],end[0],end[1],u_turn_allowed=True)
    routeLonLats = route_response["routes"][0]["geometry"]["coordinates"]
    routeLatLons = [item[::-1] for item in routeLonLats]
    dist = route_response["routes"][0]["distance"]
    time = int(np.ceil(route_response["routes"][0]["duration"]/60))
    return routeLatLons, dist, time

def plot_map(csv_data, color="blue"):
    total_distance = 0
    total_time = 0
    map_center = [1.281651,103.829894]
    map_ = folium.Map(location=map_center, zoom_start=12, tiles=None)
    folium.TileLayer("cartodb positron").add_to(map_)

    # cmap = matplotlib.colormaps['hsv']
    # point_colors = [matplotlib.colors.rgb2hex(cmap(c)) for c in points['Network_Cluster']/max(points['Network_Cluster'])]
    
    # for j in points["Network_Cluster"].unique():
    #         folium.CircleMarker(location=cluster_midpoints[j], popup="Cluster_"+str(j)+"_"+str(cluster_load[j])+"_"+str(cluster_tw[j])+"_"+str(cluster_restrictions[j]), color="black", fill=True, radius=15, opacity=0.3).add_to(map_)

    # for idx in points.index:
    #     folium.CircleMarker(location=points.loc[idx,"Node_Latlong"], popup=str(points.loc[idx,"Network_Cluster"])+"-"+str(points.loc[idx,"House"])+"_"+str(points.loc[idx,"Street"])+"_"+str(idx)+"_"+str(points.loc[idx,'Vehicle_Type_Allowed'])+"_"+str(points.loc[idx,'Tonnage_kg']), color=point_colors[idx], fill=True, radius=5).add_to(map_)
    
    for i in range(0,len(csv_data)-1):
        point = points.loc[points['id'] == csv_data[i]]
        next_point = points.loc[points['id'] == csv_data[i + 1]]
        
        folium.CircleMarker(location=point["Node_Latlong"].iloc[0], popup=str(point["Network_Cluster"].iloc[0])+"-"+str(point["House"].iloc[0])+"_"+str(point["Street"].iloc[0])+"_"+str(point['id'].iloc[0])+"_"+str(point['Vehicle_Type_Allowed'].iloc[0])+"_"+str(point['Tonnage_kg'].iloc[0]), color='red', fill=True, radius=2).add_to(map_)
        route_name = str(point['id'].iloc[0])+"-"+str(next_point['id'].iloc[0])
        route_layer = folium.FeatureGroup(name=route_name+" "+str(point["House"].iloc[0])+"_"+str(point["Street"].iloc[0])+"-"+str(next_point["House"].iloc[0])+"_"+str(next_point["Street"].iloc[0]), show=False).add_to(map_)
        routeLatLons, dist, time = find_route(point['Node_Latlong'].iloc[0], next_point['Node_Latlong'].iloc[0])
        total_distance += dist 
        total_time += time
        if routeLatLons==None:
            pass
        else:
            path = folium.PolyLine(routeLatLons,color=color,popup="dist:"+str(dist)+"_time:"+str(time)).add_to(route_layer)
            add_arrows(route_layer, path)
    print("Total_distance_taken:",total_distance)
    print("Total_time_taken:",total_time)

    folium.LayerControl().add_to(map_)
    return map_, total_distance, total_time
    
def plot_route_for_truck_number(truck_number_plate):
    csv_data = []
    for id in total_assignment[truck_number_plate]["points_id"]:
        if id == 'DP':
            csv_data.append(501)
        elif id == 'IP':
            csv_data.append(500)
        else:
            csv_data.append(int(id))
    print("Truck Number Plate: ", truck_number_plate)
    map_, dist, time = plot_map(csv_data)
    map_.save(f'{truck_number_plate}_route.html')


In [94]:
DP = {
    'id' : 501,
    'House' : 'Tuas',
    'Street' : 'Depot',
    'Vehicle_Type_Allowed' : 0,
    'Tonnage_kg' : 0,
    'Node_Latlong' : (1.3346, 103.6457),
    'Network_Cluster' : -1
}

IP = {
    'id' : 500,
    'House' : 'Tuas',
    'Street' : 'IP',
    'Vehicle_Type_Allowed' : 0,
    'Tonnage_kg' : 0,
    'Node_Latlong' : (1.296, 103.6205),
    'Network_Cluster' : -1
}

special_points_df = pd.DataFrame([IP, DP])
points = pd.concat([points, special_points_df], ignore_index = True)
points.tail()

,House,Street,Alias,Postal_Code,Premises,Direct Collection,Allowed_Veh,Ideal_Vehicle_Type,Service_Time_min,Vehicle_Type_Allowed,...,Node_Latlong,User_LongLat,Poly_Cluster,Network_Cluster,Network_Vehicle_Type_Allowed,User_Timing,Cluster_Start,Cluster_End,Completed,id
496,64,COMMONWEALTH DRIVE,None,None,HDB/Trade,Y,8,2.0,4.0,"8x2,6x4,4x2",...,"(1.299433, 103.797076)","(103.7971861717, 1.2993562266)",987.0,492,8,"(6, 19)",6.0,19.0,True,NaN
497,1,COMMONWEALTH DRIVE,BLEESED SACRAMENT CHURCH,None,Trade,Y,8,2.0,4.0,"8x2,6x4,4x2",...,"(1.295739, 103.79896)","(103.7989636894, 1.2957496234)",987.0,493,8,"(6, 19)",6.0,19.0,False,NaN
498,3,COMMONWEALTH DRIVE,SRI MUNEES-WARAN TP,None,Trade,Y,8,2.0,4.0,"8x2,6x4,4x2",...,"(1.295978, 103.798462)","(103.7985892673, 1.2958908083)",987.0,494,8,"(6, 19)",6.0,19.0,False,NaN
499,Tuas,IP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,"(1.296, 103.6205)",NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,500.0
500,Tuas,Depot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,"(1.3346, 103.6457)",NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,501.0
